# SQL Queries - Merging animal outcome data with animal intake data for three different types of breed combinations


### Imports

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
engine_name = 'postgresql://credentials/project3'
cnx = create_engine(engine_name)

In [3]:
pd.read_sql_query('''SELECT * FROM animal_outcomes LIMIT 5''',cnx)

,index,animalid,name,datetime,monthyear,dateofbirth,outcometype,outcomesubtype,animaltype,sexuponoutcome,ageuponoutcome,breed,color
0,0,A658751,Benji,2016-11-13 01:38:00,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,None,Dog,Neutered Male,60.0,Border Terrier Mix,Tan
1,1,A746650,Rose,2017-04-07 11:58:00,04/07/2017 11:58:00 AM,04/06/2016,Return to Owner,None,Dog,Intact Female,12.0,Jack Russell Terrier/Labrador Retriever,Yellow
2,2,A750122,Happy Camper,2017-05-24 06:36:00,05/24/2017 06:36:00 PM,04/08/2017,Transfer,Partner,Dog,Intact Male,1.0,Labrador Retriever Mix,Black
3,3,A666430,Lucy,2013-11-07 11:47:00,11/07/2013 11:47:00 AM,11/06/2012,Transfer,Partner,Dog,Spayed Female,12.0,Beagle Mix,Twocolor
4,4,A675708,*Johnny,2014-06-03 02:20:00,06/03/2014 02:20:00 PM,03/31/2013,Adoption,None,Dog,Neutered Male,12.0,Pit Bull,Twocolor


In [4]:
outcomes = pd.read_sql_query('''SELECT * FROM animal_outcomes''',cnx)

In [5]:
outcomes.head()

,index,animalid,name,datetime,monthyear,dateofbirth,outcometype,outcomesubtype,animaltype,sexuponoutcome,ageuponoutcome,breed,color
0,0,A658751,Benji,2016-11-13 01:38:00,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,None,Dog,Neutered Male,60.0,Border Terrier Mix,Tan
1,1,A746650,Rose,2017-04-07 11:58:00,04/07/2017 11:58:00 AM,04/06/2016,Return to Owner,None,Dog,Intact Female,12.0,Jack Russell Terrier/Labrador Retriever,Yellow
2,2,A750122,Happy Camper,2017-05-24 06:36:00,05/24/2017 06:36:00 PM,04/08/2017,Transfer,Partner,Dog,Intact Male,1.0,Labrador Retriever Mix,Black
3,3,A666430,Lucy,2013-11-07 11:47:00,11/07/2013 11:47:00 AM,11/06/2012,Transfer,Partner,Dog,Spayed Female,12.0,Beagle Mix,Twocolor
4,4,A675708,*Johnny,2014-06-03 02:20:00,06/03/2014 02:20:00 PM,03/31/2013,Adoption,None,Dog,Neutered Male,12.0,Pit Bull,Twocolor


In [7]:
outcomes['outcometype'].unique()

array(['Return to Owner', 'Transfer', 'Adoption', 'Euthanasia', 'Disposal',
       'Rto-Adopt', 'Died', 'Missing'], dtype=object)

### Obtain workflow for merging tables. Practice with the unaltered animal intake and outcome tables

In [9]:
merge_tables = pd.read_sql_query('''SELECT *
                                    FROM (SELECT
                                    animal_outcomes.datetime - animal_intakes.datetime AS "TIME_DIFF",
                                    animal_outcomes.animalid as "ID",
                                    animal_outcomes.name as "NAME",
                                    animal_outcomes.datetime as "OUTCOME_TIME",
                                    animal_outcomes.outcometype as "OUTCOME_TYPE",
                                    animal_outcomes.outcomesubtype as "OUTCOME_SUBTYPE",
                                    animal_outcomes.sexuponoutcome as "SEX_ON_OUTCOME",
                                    animal_outcomes.ageuponoutcome "AGE_ON_OUTCOME",
                                    animal_outcomes.breed as "BREED",
                                    animal_outcomes.color as "COLOR",
                                    animal_intakes.datetime as "INTAKE_TIME",
                                    animal_intakes.intaketype as "INTAKE_TYPE",
                                    animal_intakes.intakecondition as "INTAKE_CONDITION",
                                    animal_intakes.sexuponintake as "SEX_ON_INTAKE",
                                    animal_intakes.ageuponintake as "AGE_ON_INTAKE",
                                    row_number() over(PARTITION BY animal_outcomes.animalid
                                    ORDER BY abs(extract(epoch from (animal_outcomes.datetime - animal_intakes.datetime)))) AS rn
                                    FROM
                                        animal_outcomes
                                    JOIN
                                        animal_intakes
                                    ON 
                                        animal_outcomes.animalid = animal_intakes.animalid 
                                    WHERE animal_outcomes.datetime > animal_intakes.datetime) AS derivedtable
                                    WHERE rn=1''',cnx)

In [10]:
merge_tables.head()

,TIME_DIFF,ID,NAME,OUTCOME_TIME,OUTCOME_TYPE,OUTCOME_SUBTYPE,SEX_ON_OUTCOME,AGE_ON_OUTCOME,BREED,COLOR,INTAKE_TIME,INTAKE_TYPE,INTAKE_CONDITION,SEX_ON_INTAKE,AGE_ON_INTAKE,rn
0,0 days 18:14:00,A006100,Scamp,2014-12-20 04:35:00,Return to Owner,None,Neutered Male,84.0,Spinone Italiano Mix,Twocolor,2014-12-19 10:21:00,Public Assist,Normal,Neutered Male,84.0,1
1,4 days 23:17:00,A047759,Oreo,2014-04-07 03:12:00,Transfer,Partner,Neutered Male,120.0,Dachshund,Tricolor,2014-04-02 03:55:00,Owner Surrender,Normal,Neutered Male,120.0,1
2,0 days 02:52:00,A134067,Bandit,2013-11-16 11:54:00,Return to Owner,None,Neutered Male,192.0,Shetland Sheepdog,Twocolor,2013-11-16 09:02:00,Public Assist,Injured,Neutered Male,192.0,1
3,1 days 08:54:00,A141142,Bettie,2013-11-17 11:40:00,Return to Owner,None,Spayed Female,180.0,Labrador Retriever/Pit Bull,Twocolor,2013-11-16 02:46:00,Stray,Aged,Spayed Female,180.0,1
4,0 days 04:17:00,A163459,Sasha,2014-11-14 07:28:00,Return to Owner,None,Intact Female,180.0,Miniature Schnauzer Mix,Twocolor,2014-11-14 03:11:00,Stray,Normal,Intact Female,180.0,1


In [11]:
merge_tables[merge_tables['ID'] == 'A552088']

,TIME_DIFF,ID,NAME,OUTCOME_TIME,OUTCOME_TYPE,OUTCOME_SUBTYPE,SEX_ON_OUTCOME,AGE_ON_OUTCOME,BREED,COLOR,INTAKE_TIME,INTAKE_TYPE,INTAKE_CONDITION,SEX_ON_INTAKE,AGE_ON_INTAKE,rn
608,202 days 23:37:00,A552088,Sasha,2015-04-03 06:28:00,Adoption,None,Spayed Female,72.0,Labrador Retriever Mix,Brown,2014-09-12 06:51:00,Stray,Normal,Spayed Female,60.0,1


In [13]:
merge_tables[merge_tables['ID'] == 'A720298']

,TIME_DIFF,ID,NAME,OUTCOME_TIME,OUTCOME_TYPE,OUTCOME_SUBTYPE,SEX_ON_OUTCOME,AGE_ON_OUTCOME,BREED,COLOR,INTAKE_TIME,INTAKE_TYPE,INTAKE_CONDITION,SEX_ON_INTAKE,AGE_ON_INTAKE,rn
21772,4 days 04:04:00,A720298,Bully,2016-02-10 05:28:00,Return to Owner,None,Neutered Male,3.0,Pit Bull Mix,Twocolor,2016-02-06 01:24:00,Stray,Normal,Intact Male,3.0,1


In [14]:
len(merge_tables['ID'].unique())

34043

### Pickle dataframe

In [16]:
path1 = '/Users/murdock/Documents/metis/project3/merged_animals.pkl'
merge_tables.to_pickle(path1)

### Merge Purebred vs. Not Purebred Dataframes

In [17]:
purevsmix_tables = pd.read_sql_query('''SELECT *
                                        FROM (SELECT
                                        purevsmix_outcomes.datetime - animal_intakes.datetime AS "TIME_DIFF",
                                        purevsmix_outcomes.animalid as "ID",
                                        purevsmix_outcomes.name as "NAME",
                                        purevsmix_outcomes.datetime as "OUTCOME_TIME",
                                        purevsmix_outcomes.outcometype as "OUTCOME_TYPE",
                                        purevsmix_outcomes.outcomesubtype as "OUTCOME_SUBTYPE",
                                        purevsmix_outcomes.sexuponoutcome as "SEX_ON_OUTCOME",
                                        purevsmix_outcomes.ageuponoutcome "AGE_ON_OUTCOME",
                                        purevsmix_outcomes.breed as "BREED",
                                        purevsmix_outcomes.color as "COLOR",
                                        animal_intakes.datetime as "INTAKE_TIME",
                                        animal_intakes.intaketype as "INTAKE_TYPE",
                                        animal_intakes.intakecondition as "INTAKE_CONDITION",
                                        animal_intakes.sexuponintake as "SEX_ON_INTAKE",
                                        animal_intakes.ageuponintake as "AGE_ON_INTAKE",
                                        row_number() over(PARTITION BY purevsmix_outcomes.animalid
                                        ORDER BY abs(extract(epoch from (purevsmix_outcomes.datetime - animal_intakes.datetime)))) AS rn
                                        FROM
                                            purevsmix_outcomes
                                        JOIN
                                            animal_intakes
                                        ON 
                                            purevsmix_outcomes.animalid = animal_intakes.animalid
                                        WHERE purevsmix_outcomes.datetime > animal_intakes.datetime) AS derivedtable
                                        WHERE rn=1''',cnx)

In [18]:
purevsmix_tables.sample(10)

,TIME_DIFF,ID,NAME,OUTCOME_TIME,OUTCOME_TYPE,OUTCOME_SUBTYPE,SEX_ON_OUTCOME,AGE_ON_OUTCOME,BREED,COLOR,INTAKE_TIME,INTAKE_TYPE,INTAKE_CONDITION,SEX_ON_INTAKE,AGE_ON_INTAKE,rn
7815,0 days 00:41:00,A679029,None,2014-05-15 03:54:00,Euthanasia,Suffering,Intact Male,10.0,Mixed Breed,Twocolor,2014-05-15 03:13:00,Stray,Sick,Intact Male,10.000000,1
8757,51 days 18:57:00,A681970,*Allie,2014-08-13 05:44:00,Adoption,Foster,Spayed Female,1.0,Mixed Breed,Tick,2014-06-22 10:47:00,Stray,Normal,Intact Female,0.133333,1
17595,10 days 18:15:00,A707494,Karamel,2015-07-24 04:06:00,Return to Owner,None,Intact Female,108.0,Mixed Breed,Twocolor,2015-07-13 09:51:00,Public Assist,Normal,Intact Female,108.000000,1
14229,13 days 06:31:00,A697261,Roltan,2015-10-24 12:00:00,Adoption,None,Neutered Male,11.0,Mixed Breed,Twocolor,2015-10-11 05:29:00,Stray,Normal,Neutered Male,10.000000,1
21459,88 days 09:23:00,A719504,*Eagle,2016-04-18 12:00:00,Transfer,Partner,Neutered Male,36.0,Mixed Breed,Twocolor,2016-01-21 02:37:00,Stray,Normal,Intact Male,36.000000,1
11986,0 days 14:46:00,A691817,Deacon,2014-11-11 01:58:00,Return to Owner,None,Neutered Male,48.0,Mixed Breed,Twocolor,2014-11-10 11:12:00,Public Assist,Normal,Neutered Male,48.000000,1
32814,1 days 19:33:00,A755608,None,2017-08-08 04:49:00,Transfer,Partner,Intact Female,1.0,Mixed Breed,Chocolate,2017-08-06 09:16:00,Stray,Normal,Intact Female,1.000000,1
5651,4 days 02:12:00,A672558,*Frida,2014-02-16 06:08:00,Adoption,None,Spayed Female,12.0,Mixed Breed,Twocolor,2014-02-12 03:56:00,Stray,Normal,Intact Female,12.000000,1
32881,4 days 16:25:00,A755873,None,2017-08-15 03:24:00,Adoption,None,Neutered Male,3.0,Purebred,Red,2017-08-10 10:59:00,Stray,Normal,Intact Male,3.000000,1
12640,7 days 03:57:00,A693552,Cooper,2014-12-17 05:31:00,Transfer,Partner,Intact Male,4.0,Mixed Breed,Twocolor,2014-12-10 01:34:00,Stray,Normal,Intact Male,4.000000,1


### Pickle Dataframe

In [19]:
path2 = '/Users/murdock/Documents/metis/project3/purevsmix_mergedtables.pkl'
purevsmix_tables.to_pickle(path2)

### Merge pitbull vs not pitbull tables

In [20]:
pitbullvsnot_tables = pd.read_sql_query('''SELECT *
                                           FROM (SELECT
                                           pitbull_outcomes.datetime - animal_intakes.datetime AS "TIME_DIFF",
                                           pitbull_outcomes.animalid as "ID",
                                           pitbull_outcomes.name as "NAME",
                                           pitbull_outcomes.datetime as "OUTCOME_TIME",
                                           pitbull_outcomes.outcometype as "OUTCOME_TYPE",
                                           pitbull_outcomes.outcomesubtype as "OUTCOME_SUBTYPE",
                                           pitbull_outcomes.sexuponoutcome as "SEX_ON_OUTCOME",
                                           pitbull_outcomes.ageuponoutcome "AGE_ON_OUTCOME",
                                           pitbull_outcomes.breed as "BREED",
                                           pitbull_outcomes.color as "COLOR",
                                           animal_intakes.datetime as "INTAKE_TIME",
                                           animal_intakes.intaketype as "INTAKE_TYPE",
                                           animal_intakes.intakecondition as "INTAKE_CONDITION",
                                           animal_intakes.sexuponintake as "SEX_ON_INTAKE",
                                           animal_intakes.ageuponintake as "AGE_ON_INTAKE",
                                           row_number() over(PARTITION BY pitbull_outcomes.animalid
                                           ORDER BY abs(extract(epoch from (pitbull_outcomes.datetime - animal_intakes.datetime)))) AS rn
                                           FROM
                                               pitbull_outcomes
                                           JOIN
                                               animal_intakes
                                           ON 
                                               pitbull_outcomes.animalid = animal_intakes.animalid
                                           WHERE pitbull_outcomes.datetime > animal_intakes.datetime) AS derivedtable
                                           WHERE rn=1''',cnx)

In [21]:
pitbullvsnot_tables.head()

,TIME_DIFF,ID,NAME,OUTCOME_TIME,OUTCOME_TYPE,OUTCOME_SUBTYPE,SEX_ON_OUTCOME,AGE_ON_OUTCOME,BREED,COLOR,INTAKE_TIME,INTAKE_TYPE,INTAKE_CONDITION,SEX_ON_INTAKE,AGE_ON_INTAKE,rn
0,0 days 18:14:00,A006100,Scamp,2014-12-20 04:35:00,Return to Owner,None,Neutered Male,84.0,Not Pit Bull,Twocolor,2014-12-19 10:21:00,Public Assist,Normal,Neutered Male,84.0,1
1,4 days 23:17:00,A047759,Oreo,2014-04-07 03:12:00,Transfer,Partner,Neutered Male,120.0,Not Pit Bull,Tricolor,2014-04-02 03:55:00,Owner Surrender,Normal,Neutered Male,120.0,1
2,0 days 02:52:00,A134067,Bandit,2013-11-16 11:54:00,Return to Owner,None,Neutered Male,192.0,Not Pit Bull,Twocolor,2013-11-16 09:02:00,Public Assist,Injured,Neutered Male,192.0,1
3,1 days 08:54:00,A141142,Bettie,2013-11-17 11:40:00,Return to Owner,None,Spayed Female,180.0,Pit Bull,Twocolor,2013-11-16 02:46:00,Stray,Aged,Spayed Female,180.0,1
4,0 days 04:17:00,A163459,Sasha,2014-11-14 07:28:00,Return to Owner,None,Intact Female,180.0,Not Pit Bull,Twocolor,2014-11-14 03:11:00,Stray,Normal,Intact Female,180.0,1


### Pickle Dataframe

In [22]:
path3 = '/Users/murdock/Documents/metis/project3/pitbull_mergedtables.pkl'
pitbullvsnot_tables.to_pickle(path3)

### Merge breeds with low occurrences (< 50)

In [23]:
lowoccur_tables = pd.read_sql_query('''SELECT *
                                       FROM (SELECT
                                       lowoccur_outcomes.datetime - animal_intakes.datetime AS "TIME_DIFF",
                                       lowoccur_outcomes.animalid as "ID",
                                       lowoccur_outcomes.name as "NAME",
                                       lowoccur_outcomes.datetime as "OUTCOME_TIME",
                                       lowoccur_outcomes.outcometype as "OUTCOME_TYPE",
                                       lowoccur_outcomes.outcomesubtype as "OUTCOME_SUBTYPE",
                                       lowoccur_outcomes.sexuponoutcome as "SEX_ON_OUTCOME",
                                       lowoccur_outcomes.ageuponoutcome "AGE_ON_OUTCOME",
                                       lowoccur_outcomes.breed as "BREED",
                                       lowoccur_outcomes.color as "COLOR",
                                       animal_intakes.datetime as "INTAKE_TIME",
                                       animal_intakes.intaketype as "INTAKE_TYPE",
                                       animal_intakes.intakecondition as "INTAKE_CONDITION",
                                       animal_intakes.sexuponintake as "SEX_ON_INTAKE",
                                       animal_intakes.ageuponintake as "AGE_ON_INTAKE",
                                       row_number() over(PARTITION BY lowoccur_outcomes.animalid
                                       ORDER BY abs(extract(epoch from (lowoccur_outcomes.datetime - animal_intakes.datetime)))) AS rn
                                       FROM
                                           lowoccur_outcomes
                                       JOIN
                                           animal_intakes
                                       ON 
                                           lowoccur_outcomes.animalid = animal_intakes.animalid
                                       WHERE lowoccur_outcomes.datetime > animal_intakes.datetime) AS derivedtable
                                       WHERE rn=1''',cnx)

In [24]:
lowoccur_tables.head()

,TIME_DIFF,ID,NAME,OUTCOME_TIME,OUTCOME_TYPE,OUTCOME_SUBTYPE,SEX_ON_OUTCOME,AGE_ON_OUTCOME,BREED,COLOR,INTAKE_TIME,INTAKE_TYPE,INTAKE_CONDITION,SEX_ON_INTAKE,AGE_ON_INTAKE,rn
0,0 days 18:14:00,A006100,Scamp,2014-12-20 04:35:00,Return to Owner,None,Neutered Male,84.0,Other Mixed Breed,Twocolor,2014-12-19 10:21:00,Public Assist,Normal,Neutered Male,84.0,1
1,4 days 23:17:00,A047759,Oreo,2014-04-07 03:12:00,Transfer,Partner,Neutered Male,120.0,Dachshund,Tricolor,2014-04-02 03:55:00,Owner Surrender,Normal,Neutered Male,120.0,1
2,0 days 02:52:00,A134067,Bandit,2013-11-16 11:54:00,Return to Owner,None,Neutered Male,192.0,Other Purebred,Twocolor,2013-11-16 09:02:00,Public Assist,Injured,Neutered Male,192.0,1
3,1 days 08:54:00,A141142,Bettie,2013-11-17 11:40:00,Return to Owner,None,Spayed Female,180.0,Labrador Retriever/Pit Bull,Twocolor,2013-11-16 02:46:00,Stray,Aged,Spayed Female,180.0,1
4,0 days 04:17:00,A163459,Sasha,2014-11-14 07:28:00,Return to Owner,None,Intact Female,180.0,Miniature Schnauzer Mix,Twocolor,2014-11-14 03:11:00,Stray,Normal,Intact Female,180.0,1


### Pickle Dataframe

In [25]:
path4 = '/Users/murdock/Documents/metis/project3/lowoccur_mergedtables.pkl'
lowoccur_tables.to_pickle(path4)